In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time

import sionna

# Import Sionna RT components
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, Camera, Paths2CIR

# For link-level simulations
from sionna.channel import cir_to_ofdm_channel, subcarrier_frequencies, OFDMChannel, ApplyOFDMChannel, CIRDataset
from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver
from sionna.utils import compute_ber, ebnodb2no, PlotBER
from sionna.ofdm import KBestDetector, LinearDetector
from sionna.mimo import StreamManagement

In [ ]:
# Load integrated scene
scene = load_scene(sionna.rt.scene.simple_street_canyon) # Try also sionna.rt.scene.etoile

# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=1, 
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="tr38901",
                             polarization="V")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="dipole",
                             polarization="cross")

center = scene.center
tx_position = [center[0], center[1] + 15, 55]
rx_position = [center[0], center[1], 1.2]

# Create transmitter
tx = Transmitter(name="tx",
                 position=tx_position)

# Add transmitter instance to scene
scene.add(tx)

# Create a receiver
rx = Receiver(name="rx",
              position=rx_position,
              orientation=[0,0,0])

# Add receiver instance to scene
scene.add(rx)

tx.look_at(rx) # Transmitter points towards receiver

scene.frequency = 2.14e9 # in Hz; implicitly updates RadioMaterials

scene.synthetic_array = True # If set to False, ray tracing will be done per antenna element (slower for large arrays)

scene.preview()

In [ ]:
# Compute propagation paths
paths = scene.compute_paths(max_depth=5,
                            method="stochastic", # For small scenes the method can be also set to "exhaustive"
                            num_samples=1e6,     # Number of rays shot into random directions, too few rays can lead to missing paths
                            seed=42)             # By fixing the seed, reproducible results can be ensured

scene.preview(paths, show_devices=True, show_paths=True) # Use the mouse to focus on the visualized paths